In [18]:
import torch
import torchvision
import pandas as pd
from util.util import My_Transform, My_Normalize, metric
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
from PIL import Image
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch_geometric.nn import global_mean_pool, avg_pool

In [13]:
import numpy as np
import torch.nn as nn
import torch
import math
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
from PIL import Image
import torchvision.transforms as transforms


class GANLoss(nn.Module):
    """ Initialize the GANLoss class.
    Parameters:
        gan_mode (str) - - the type of GAN objective. It currently supports vanilla, lsgan, and wgangp.
        target_real_label (bool) - - label for a real image
        target_fake_label (bool) - - label of a fake image
    Note: Do not use sigmoid as the last layer of Discriminator.
    LSGAN needs no sigmoid. vanilla GANs will handle it with BCEWithLogitsLoss.
    """
    def __init__(self, use_lsgan=True,
                 target_real_label=1.0,
                 target_fake_label=0.0,
                 tensor=torch.FloatTensor):
        super(GANLoss, self).__init__()
        self.register_buffer('real_label', torch.tensor(target_real_label))
        self.register_buffer('fake_label', torch.tensor(target_fake_label))
        self.real_label_var = None
        self.fake_label_var = None
        self.Tensor = tensor
        if use_lsgan:
            self.loss = nn.MSELoss()
        else:
            self.loss = nn.BCEWithLogitsLoss()

    def get_target_tensor(self, input,
                          target_is_real):
        if target_is_real:
            target_tensor = self.real_label
        else:
            target_tensor = self.fake_label
        return target_tensor.expand_as(input)

    def __call__(self, input,
                 target_is_real):
        target_tensor = self.get_target_tensor(input, target_is_real)
        target_tensor = target_tensor.cuda()
        print(target_tensor.size(), target_tensor)
        # print('device check ',input.device, target_tensor.device)
        return self.loss(input, target_tensor)

def get_adj(size):
    index_matrix = [[j*size + i for i in range(size)] for j in range(size)]
    adj = []
    for x in range(0, size):
        for y in range(0, size):
            # upper row
            if x-1 >= 0:
                if y-1 >= 0:
                    adj.append([index_matrix[x][y], index_matrix[x-1][y-1]])
                adj.append([index_matrix[x][y], index_matrix[x-1][y]])
                if y+1 < size:
                    adj.append([index_matrix[x][y], index_matrix[x-1][y+1]])
            # current row
            if y-1 >= 0:
                adj.append([index_matrix[x][y], index_matrix[x][y-1]])
            # adj.append([x,y])
            if y+1 < size:
                adj.append([index_matrix[x][y], index_matrix[x][y+1]])
            # lower row
            if x+1 < size:
                if y - 1 >= 0:
                    adj.append([index_matrix[x][y], index_matrix[x + 1][y - 1]])
                adj.append([index_matrix[x][y], index_matrix[x + 1][y]])
                if y + 1 < size:
                    adj.append([index_matrix[x][y], index_matrix[x + 1][y + 1]])
        # adj.sort(key = lambda x:x[0])
    return adj

def get_similarity(size, feature_matrix):
    cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)
    wt = []
    for x in range(0, size):
        for y in range(0, size):
            # upper row
            if x-1 >= 0:
                if y-1 >= 0:
                    wt.append(cos_sim(feature_matrix[:,x,y,:], feature_matrix[:,x-1,y-1,:]).tolist())
                wt.append(cos_sim(feature_matrix[:, x, y, :], feature_matrix[:, x - 1, y, :]).tolist())
                if y+1 < size:
                    wt.append(cos_sim(feature_matrix[:, x, y, :], feature_matrix[:, x - 1, y + 1, :]).tolist())
            # current row
            if y-1 >= 0:
                wt.append(cos_sim(feature_matrix[:, x, y, :], feature_matrix[:, x, y - 1, :]).tolist())
            # adj.append([x,y])
            if y+1 < size:
                wt.append(cos_sim(feature_matrix[:, x, y, :], feature_matrix[:, x, y + 1, :]).tolist())
            # lower row
            if x+1 < size:
                if y - 1 >= 0:
                    wt.append(cos_sim(feature_matrix[:, x, y, :], feature_matrix[:, x+1, y - 1, :]).tolist())
                wt.append(cos_sim(feature_matrix[:, x, y, :], feature_matrix[:, x + 1, y, :]).tolist())
                if y + 1 < size:
                    wt.append(cos_sim(feature_matrix[:, x, y, :], feature_matrix[:, x + 1, y + 1, :]).tolist())
    wt = torch.tensor(wt) # number_edge, batch
    n_e, n_b = wt.size()
    wt = wt.view(n_b, n_e) # batch, number_edge
    return wt

def get_idx(data, num_labelled):
    labels = data.y

    # obtain train/val/test idx - ignore val for now
    label_pos = (labels == 1).nonzero(as_tuple=True)[0]
    label_neg = (labels == 0).nonzero(as_tuple=True)[0]
    print('total positive ',len(label_pos))
    print('total negative ', len(label_neg))
    neg2pos = len(label_neg)/len(label_pos)
    # select train idx
    index_pos_train = label_pos.float().multinomial(num_samples=num_labelled, replacement=True)
    index_neg_train = label_neg.float().multinomial(num_samples=num_labelled, replacement=True)
    index_train = torch.cat((label_pos[index_pos_train].long(), label_neg[index_neg_train].long()), 0)
    # use the rest as test idx
    label_pos_test = []
    label_neg_test = []
    for i in label_pos:
        if i not in label_pos[index_pos_train]:
            label_pos_test.append(i)
    for i in label_neg:
        if i not in label_neg[index_neg_train]:
            label_neg_test.append(i)

    index_test = torch.cat((torch.tensor(label_pos_test).long(), torch.tensor(label_neg_test).long()), 0)
    return index_train, index_test, neg2pos

def obtain_identity_adj(data):
    n = data.size()[0]
    source, target = np.arange(n).astype(int), np.arange(n).astype(int)
    adj_edge_list = np.concatenate(([source], [target]), axis=0)
    adj_edge_list = torch.tensor(adj_edge_list, dtype=torch.long).contiguous()
    return adj_edge_list

def accuracy(output, labels, detail=False):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    if detail:
        print('preds pos', len(preds[preds == 1]))
        print('preds neg', len(preds[preds == 0]))
        print('number pos ', len(labels[labels == 1]))
        print('number neg ', len(labels[labels == 0]))
        correct_pos = 0
        correct_neg = 0
        for i in range(len(labels)):
            if labels[i] == preds[i] == 0:
                correct_neg += 1
            elif labels[i] == preds[i] == 1:
                correct_pos += 1
            else:
                continue
        print('number of correct positive', correct_pos)
        print('number of correct negative', correct_neg)
        print()
    return correct / len(labels)

def adaptation_factor(x):
    den = 1.0 + math.exp(-10 * x)
    lamb = 2.0 / den - 1.0
    return min(lamb, 1.0)

class My_Transform(object):
    '''
    customised image augmentations
    '''
    def __init__(self):
        pass

    def __call__(self, sample):
        image, label, pth = sample['image'], sample['label'], sample['pth']
        aug = transforms.Compose([
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomVerticalFlip(0.5),
            # transforms.RandomRotation(degrees=(0, 180)),
            # transforms.RandomPosterize(bits=2),
            # transforms.RandomAdjustSharpness(sharpness_factor=2),
            # transforms.RandomAutocontrast()
        ])
        augmented = aug(image)

        return {'image': augmented, 'label':label, 'pth':pth}

class My_Normalize(object):
    '''
    Normalise image data
    '''
    def __init__(self):
        pass

    def __call__(self, sample):
        image, label, pth = sample['image'], sample['label'], sample['pth']
        aug_list = [transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                    ]
        aug = transforms.Compose(aug_list)
        # print(image) torch.FloatTensor([label])
        augmented = aug(image)
        return {'image': augmented, 'label':label, 'pth':pth}


def metric(output, labels, batch_size, use_graph=False, use_pool=False):
    p_label = {}
    TP = TN = FP = FN = 0
    slide_label = []
    slide_prob = []

    if use_graph and not use_pool:
        use_patch_prediction = True
    else:
        use_patch_prediction = False

    if use_patch_prediction:
        output = output.view(batch_size, -1, 2)
        for i in range(output.size()[0]):
            prediction_matrix = torch.argmax(output[i], dim=1)
            # prediction = torch.mean(prediction_matrix.float()).int()
            prediction = 1 if 1 in prediction_matrix else 0
            label_use = torch.argmax(labels[i]).cpu().item()
            if prediction == label_use == 0:
                TN += 1
            elif prediction == label_use == 1:
                TP += 1
            elif prediction == 0 and label_use == 1:
                FN += 1
            else:
                FP += 1
    else:
        for i in range(output.size()[0]):
            prediction = torch.argmax(output[i])
            label_use = torch.argmax(labels[i]).cpu().item()
            if prediction.cpu().item() == label_use == 0:
                TN += 1
            elif prediction.cpu().item() == label_use == 1:
                TP += 1
            elif prediction.cpu().item() == 0 and label_use == 1:
                FN += 1
            else:
                FP += 1
            slide_label.append(label_use)
            slide_prob.append(output[i][1].cpu().item())
    return TN, TP, FN, FP

In [14]:
class WSIDataset(Dataset):
    """Generate dataset."""
    def __init__(self, filepath, crf=False, annotation_pth=False):
        self.all_data = pd.read_csv(filepath)
        self.crf = crf
        if not self.crf:
            all_imgpth = self.all_data['image'].tolist()
            all_label = self.all_data['label'].tolist()
            self.X_train, self.X_test, self.y_train, self.y_test = self.stage_split(all_imgpth, all_label)
            self.aug_transform = transforms.Compose([My_Transform()])
            self.normalise_transform = transforms.Compose([My_Normalize()])
        else:
            self.annotation = annotation_pth
            all_normal, all_tumor = self.Obtain_detail(self.all_data['label'].tolist())
            print('total: ', all_normal, all_tumor)
            self.train_data = self.all_data.sample(frac=0.8, replace=False, random_state=200)  # random state is a seed value
            self.test_data = self.all_data.drop(self.train_data.index)
            train_normal, train_tumor = self.Obtain_detail(self.train_data['label'].tolist())
            print('train: ', train_normal, train_tumor)
            test_normal, test_tumor = self.Obtain_detail(self.test_data['label'].tolist())
            print('test: ', test_normal, test_tumor)

    def stage_split(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
        # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)  # 0.25 x 0.8 = 0.2
        print('Train detail: ')
        train_normal, train_tumor = self.Obtain_detail(y_train)
        print('normal vs tumor = %d vs %d'%(train_normal, train_tumor))
        print('Test detail: ')
        test_normal, test_tumor = self.Obtain_detail(y_test)
        print('normal vs tumor = %d vs %d'%(test_normal, test_tumor))
        return X_train, X_test, y_train, y_test

    def Obtain_detail(self, y):
        normal = tumor = 0
        for i in y:
            if i == 'Normal':
                normal += 1
            else:
                tumor += 1
        return normal, tumor

    def Obtain_dataset(self, stage):
        if stage == 'Train':
            if self.crf:
                self.dataset = GridImageDataset(self.train_data, self.annotation, 768, 256)
            else:
                self.dataset = StageDataset(self.X_train, self.y_train, self.normalise_transform, augmentation=self.aug_transform)
        elif stage == 'Test':
            if self.crf:
                self.dataset = GridImageDataset(self.test_data, self.annotation, 768, 256)
            else:
                self.dataset = StageDataset(self.X_test, self.y_test, self.normalise_transform)
        return self.dataset

    def Obtain_loader(self, stage, batch_size):
        ds = self.Obtain_dataset(stage)
        self.loader = DataLoader(ds,
                                 batch_size=batch_size,
                                 shuffle=True,
                                 num_workers=1,
                                 drop_last=True)
        return self.loader


class StageDataset(Dataset):
    def __init__(self, X, y, normalisation, augmentation=False):
        self.map_dict = {'Normal': torch.FloatTensor([1.0, 0.0]), 'Tumor': torch.FloatTensor([0.0, 1.0])}
        self.X = X
        self.y = y
        self.normalisation = normalisation
        self.augmentation = augmentation

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        '''image	label'''
        img_path = self.X[idx]
        label = self.map_dict[self.y[idx]]
        # image = cv2.imread(img_path, cv2.COLOR_BGR2RGB)
        image = Image.open(img_path)
        sample = {'image': image, 'label': label, 'pth':img_path}

        if self.augmentation:
            sample = self.augmentation(sample)
        sample = self.normalisation(sample)
        return sample


class GridImageDataset(Dataset):
    """
    Data producer that generate a square grid, e.g. 3x3, of patches and their
    corresponding labels from pre-sampled images.
    """

    def __init__(self, data_path, json_path, img_size, patch_size,
                 crop_size=224, normalize=True):
        """
        Initialize the data producer.
        Arguments:
            data_path: string, path to pre-sampled images using patch_gen.py
            img_size: int, size of pre-sampled images, e.g. 768
            patch_size: int, size of the patch, e.g. 256
            crop_size: int, size of the final crop that is feed into a CNN,
                e.g. 224 for ResNet
            normalize: bool, if normalize the [0, 255] pixel values to [-1, 1],
                mostly False for debuging purpose
        """
        self._df = data_path
        self._json_path = json_path
        self._img_size = img_size
        self._patch_size = patch_size
        self._crop_size = crop_size
        self._normalize = normalize
        self._color_jitter = transforms.ColorJitter(64.0 / 255, 0.75, 0.25, 0.04)
        self._preprocess()

    def _preprocess(self):
        if self._img_size % self._patch_size != 0:
            raise Exception('Image size / patch size != 0 : {} / {}'.
                            format(self._img_size, self._patch_size))

        self._patch_per_side = self._img_size // self._patch_size
        self._grid_size = self._patch_per_side * self._patch_per_side

        self._pids = list(map(lambda x: x.strip('.json'),
                              os.listdir(self._json_path)))


    def __len__(self):
        return len(self._df)

    def __getitem__(self, idx):
        image_pth = self._df.iloc[idx, 0]
        mask_pth = self._df.iloc[idx, 3]

        slide_name = self._df.iloc[idx, 2]
        image_name = image_pth.split('/')[-1].split('.png')[0].split('_')
        x_top_left, y_top_left = 0, 0  # int(image_name[0]), int(image_name[1])
        patch_label = self._df.iloc[idx, 1]

        # the grid of labels for each patch
        label_grid = np.zeros((self._patch_per_side, self._patch_per_side),
                              dtype=np.float32)
        if mask_pth != 'Nothing':
            mask_np = cv2.imread(mask_pth)
            for x_i in range(self._patch_per_side):
                x_t = x_top_left + self._patch_size * x_i
                for y_i in range(self._patch_per_side):
                    y_t = y_top_left + self._patch_size * y_i
                    select_tumor_mask = mask_np[x_t: x_t + self._patch_size, y_t: y_t + self._patch_size]
                    include_tumor = np.count_nonzero(select_tumor_mask)

                    if include_tumor / (mask_np.shape[0] * mask_np.shape[1]) > 0.001:
                        label = 1
                    else:
                        label = 0

                    label_grid[y_i, x_i] = label

        img = Image.open(image_pth)

        # color jitter
        img = self._color_jitter(img)

        # use left_right flip
        if np.random.rand() > 0.5:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
            label_grid = np.fliplr(label_grid)

        # use rotate
        num_rotate = np.random.randint(0, 4)
        img = img.rotate(90 * num_rotate)
        label_grid = np.rot90(label_grid, num_rotate)

        # PIL image:   H x W x C
        # torch image: C X H X W
        img = np.array(img, dtype=np.float32).transpose((2, 0, 1))

        if self._normalize:
            img = (img - 128.0) / 128.0

        # flatten the square grid
        img_flat = np.zeros(
            (self._grid_size, 3, self._crop_size, self._crop_size),
            dtype=np.float32)
        label_flat = np.zeros(self._grid_size, dtype=np.float32)

        idx = 0
        for x_idx in range(self._patch_per_side):
            for y_idx in range(self._patch_per_side):
                # center crop each patch
                x_start = int(
                    (x_idx + 0.5) * self._patch_size - self._crop_size / 2)
                x_end = x_start + self._crop_size
                y_start = int(
                    (y_idx + 0.5) * self._patch_size - self._crop_size / 2)
                y_end = y_start + self._crop_size
                img_flat[idx] = img[:, x_start:x_end, y_start:y_end]
                label_flat[idx] = label_grid[x_idx, y_idx]

                idx += 1

        return (img_flat, label_flat)

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch_geometric.nn import GCNConv, GATConv, GATv2Conv, TransformerConv, \
    TAGConv, SGConv, HypergraphConv, ClusterGCNConv, GENConv, FiLMConv, SuperGATConv, \
    HGTConv

class DenseGNN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, batch_idx='No'):
        super(DenseGNN, self).__init__()
        base_model = torchvision.models.densenet121()#torchvision.models.resnet50()
        self.feature_et = nn.Sequential(*list(base_model.children())[:-1])
        self.gcn1 = GATConv(nfeat, nhid)
        self.gcn2 = GATConv(nhid, nclass)
        self.dropout = dropout
        self.MSEloss =  nn.MSELoss()
        self.batch_idx = batch_idx

    def forward(self, x, graph_data):
        # get deep feature
        x = self.feature_et(x)
        bn, ch, h, w = x.shape

        # get gcn feature
        x = x.view(bn * h * w, ch)
        adj = graph_data.edge_index
        x_gcn = F.relu(self.gcn1(x, adj))
        # x_gcn = F.dropout(x_gcn, self.dropout, training=self.training)
        x_gcn = self.gcn2(x_gcn, adj)
        gcn_features = x_gcn
        if not isinstance(self.batch_idx, str):
            gcn_pool = global_mean_pool(gcn_features, self.batch_idx)
            return gcn_pool

        return gcn_features

    def criterion(self, input, label, bn):
        if not isinstance(self.batch_idx, str):
            '''
            no pooling, expand lable to matrix and apply patch-loss
            '''
            n_repeat = int(input.size()[0] / bn)
            label = torch.repeat_interleave(label, repeats=n_repeat, dim=0).float()

        return torch.nn.BCEWithLogitsLoss()(input, label)


In [19]:
from omegaconf import DictConfig
import hydra
import time
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import copy
import os
import torch
from util.util import metric, get_adj
from data.wsi_dataset import WSIDataset
import torchvision
from torch_geometric.data import Data, DataLoader

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


def runner():
    feature1 = 1024
    feature2 = 512
    data_root = '/home/congz3414050/HistoGCN/data/5X/all_data.csv'
    batch_size = 32
    num_epochs = 60
    use_graph = True
    use_pool = True
    model_save_path = '/home/congz3414050/HistoGCN/checkpoint/Scratch_Dense50GATpool_torch.pt'
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print('\n Using : ', device)

    wsi_dataset = WSIDataset(data_root)
    trainloader = wsi_dataset.Obtain_loader('Train', batch_size)
    testloader = wsi_dataset.Obtain_loader('Test', batch_size)

    # check readed data
    print('number of training data %d' % len(trainloader))
    print('number of testing data %d' % (len(testloader)))

    adj_matrx = get_adj(8)
    adj_matrx = np.array(adj_matrx)
    adj_matrx = torch.tensor(adj_matrx, dtype=torch.long)
    adj_matrx = adj_matrx.t().contiguous()
    data_list = []
    for i in range(batch_size):
        data_list.append(Data(x=torch.rand(1, 1), edge_index=adj_matrx))
    graphloader = DataLoader(data_list, batch_size=batch_size)
    batch_idx = torch.tensor([i for i in range(batch_size)])
    batch_idx = batch_idx.unsqueeze(-1)
    batch_idx = torch.repeat_interleave(batch_idx, repeats=64, dim=0).float()
    batch_idx = batch_idx.squeeze(-1)
    batch_idx = batch_idx.long().cuda()

    # define model
    if use_graph:
        print('using cnn+gcn')
        if use_pool:
            model = DenseGNN(feature1, feature2, nclass=2, dropout=0.5, batch_idx=batch_idx)
        else:
            model = DenseGNN(feature1, feature2, nclass=2, dropout=0.5)
        model.to(device)
    else:
        print('using cnn')
        model = torchvision.models.densenet121()
        set_parameter_requires_grad(model, False)

        # num_ftrs = model.fc.in_features
        # model.fc = nn.Linear(num_ftrs, 2)

        model.classifier = nn.Linear(1024, 2)
        model.to(device)


    since = time.time()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)  # , weight_decay=0.01)

    criterion = nn.BCEWithLogitsLoss()
    best_acc = 0.0
    best_loss = float('inf')
    best_f1 = 0.0
    best_epoch = 0

    for epoch in range(num_epochs):
        dataloaders = {'train': trainloader, 'val': testloader}
        batch_TP = 0
        batch_TN = 0
        batch_FP = 0
        batch_FN = 0
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode
            running_loss = 0.0

            # Iterate over data.
            for data in tqdm(dataloaders[phase]):
                inputs = data['image'].to(device=device, dtype=torch.float)
                labels = data['label'].to(device=device, dtype=torch.int64)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if use_graph:
                        for graph_data in graphloader:
                            outputs = model(inputs, graph_data.cuda())
                            labels = labels.squeeze()
                            loss = model.criterion(outputs, labels, batch_size)
                            TN, TP, FN, FP = metric(outputs, labels, batch_size, use_graph=use_graph, use_pool=use_pool)
                    else:
                        outputs = model(inputs)
                        labels = labels.squeeze()
                        loss = criterion(outputs, labels.float())
                        # wt = torch.FloatTensor([0.8, 8.0]).cuda()
                        # loss = (loss * wt).mean()
                        TN, TP, FN, FP = metric(outputs, labels, batch_size)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                batch_TN += TN
                batch_TP += TP
                batch_FN += FN
                batch_FP += FP

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            Specificity = batch_TN / (batch_TN + batch_FP)
            Sensitivity = batch_TP / (batch_FN + batch_TP)
            Precision = batch_TP / (batch_TP + batch_FP)
            F1_Score = 2 * (Precision * Sensitivity) / (Precision + Sensitivity)

            print('Stage %s'%phase)
            print('Specificity: ', Specificity)
            print('Sensitivity: ', Sensitivity)
            print('Precision: ', Precision)
            print('F1-Score: ', F1_Score)
            print('Loss: {:.4f} Lr: {}'.format(epoch_loss, optimizer.param_groups[0]["lr"]))

            # deep copy the modela
            if phase == 'val':
                val_loss = epoch_loss
                if F1_Score > best_f1:
                    best_f1 = F1_Score
                    best_epoch = epoch
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(best_model_wts, model_save_path)
                print('===========End of Epoch %d============='%epoch)
            # break
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val f1: {:4f} at epoch {:d}'.format(best_f1, best_epoch))

In [ ]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

runner()


 Using :  cuda
Train detail: 
normal vs tumor = 46353 vs 5509
Test detail: 
normal vs tumor = 11589 vs 1377
number of training data 1620
number of testing data 405
using cnn+gcn
Epoch 0/59
----------


100%|███████████████████████████████████████████████████████████████████████████████| 1620/1620 [08:09<00:00,  3.31it/s]


Stage train
Specificity:  0.9792601864640884
Sensitivity:  0.4226017441860465
Precision:  0.707636142379069
F1-Score:  0.5291775679672392
Loss: 0.2242 Lr: 0.0002


100%|█████████████████████████████████████████████████████████████████████████████████| 405/405 [00:52<00:00,  7.66it/s]


Stage val
Specificity:  0.97762392306497
Sensitivity:  0.4659206510681587
Precision:  0.7121279431363838
F1-Score:  0.5632961433716948
Loss: 0.1781 Lr: 0.0002
===========End of Epoch 0=============
Epoch 1/59
----------


...

100%|███████████████████████████████████████████████████████████████████████████████| 1620/1620 [06:13<00:00,  4.34it/s]


Stage train
Specificity:  0.9938920423887941
Sensitivity:  0.9408026148538224
Precision:  0.9482064421669106
F1-Score:  0.9444900191413726
Loss: 0.0305 Lr: 0.0002


100%|█████████████████████████████████████████████████████████████████████████████████| 405/405 [00:48<00:00,  8.36it/s]


Stage val
Specificity:  0.9933180240689262
Sensitivity:  0.897137875926195
Precision:  0.9410240780249923
F1-Score:  0.9185570844179992
Loss: 0.1304 Lr: 0.0002
===========End of Epoch 53=============
Epoch 54/59
----------


 14%|███████████▍                                                                    | 231/1620 [00:46<04:24,  5.26it/s]